In [24]:
import os

os.chdir('../')
%pwd

'd:\\Computer Vision\\Projects\\Chest Cancer Classification'

In [28]:
os.chdir('d:\Computer Vision\Projects\Chest Cancer Classification\EndToEndChestCancerClassificationUsingMlflowAndDVC')
%pwd

'd:\\Computer Vision\\Projects\\Chest Cancer Classification\\EndToEndChestCancerClassificationUsingMlflowAndDVC'

In [25]:
import dagshub
dagshub.init(repo_owner='ftask297', repo_name='EndToEndChestCancerClassificationUsingMlflowAndDVC', mlflow=True)

Initialized MLflow to track repo "ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC"

[2024-06-27 18:44:36,845: INFO: helpers: Initialized MLflow to track repo "ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC"]


Repository ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC initialized!

[2024-06-27 18:44:36,847: INFO: helpers: Repository ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC initialized!]


In [26]:
import tensorflow as tf

In [29]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [33]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [ ]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC.mlflow")


In [ ]:
import dagshub
dagshub.init(repo_owner='ftask297', repo_name='EndToEndChestCancerClassificationUsingMlflowAndDVC', mlflow=True)


Initialized MLflow to track repo "ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC"

[2024-06-27 17:22:37,179: INFO: helpers: Initialized MLflow to track repo "ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC"]


Repository ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC initialized!

[2024-06-27 17:22:37,181: INFO: helpers: Repository ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC initialized!]


In [ ]:
import requests

mlflow_server_url = "https://dagshub.com/ftask297/EndToEndChestCancerClassificationUsingMlflowAndDVC.mlflow"

try:
    response = requests.get(f"{mlflow_server_url}/api/2.0/mlflow/registered-models/list")
    if response.status_code == 200:
        print("Model registry is supported.")
    else:
        print(f"Model registry is not supported. Status code: {response.status_code}")
except Exception as e:
    print(f"An error occurred: {e}")


Model registry is not supported. Status code: 404


In [35]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-27 18:47:10,077: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-27 18:47:10,082: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-27 18:47:10,084: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 13s 2s/step - loss: 24.2417 - accuracy: 0.4314
[2024-06-27 18:47:24,055: INFO: common: json file saved at: scores.json]


2024/06/27 18:48:37 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-06-27 18:48:39,832: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpm_7gorri\model\data\model\assets
[2024-06-27 18:48:40,610: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpm_7gorri\model\data\model\assets]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/06/27 18:51:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
